<a href="https://colab.research.google.com/github/guilhermecgs/upscaler/blob/main/4k_Video_Upscaler_Colab_(Real_ESRGAN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4k Video Upscaler Colab (Real-ESRGAN)

Adapted from: [Real-ESRGAN](https://github.com/xinntao/Real-ESRGAN)

Made with ❤️ by: [yuvraj108c](https://github.com/yuvraj108c)

Github repository: https://github.com/yuvraj108c/4k-video-upscaler-colab

# 1. Setup (~1 minute)

In [11]:
import os
import subprocess
import torch

# 1. Basic Setup & Dependency Check
print("Setting up environment...")
if not os.path.exists('Real-ESRGAN'):
    !git clone https://github.com/xinntao/Real-ESRGAN.git
    %cd Real-ESRGAN
else:
    if os.getcwd().split('/')[-1] != 'Real-ESRGAN':
        %cd Real-ESRGAN

# 2. Install Dependencies (Quietly)
print("Installing dependencies...")
!pip install -q basicsr facexlib gfpgan ffmpeg ffmpeg-python
!pip install -q -r requirements.txt
!python setup.py develop > /dev/null 2>&1
!pip install "numpy<2.0" > /dev/null 2>&1

# 3. CRITICAL FIX: Patch 'basicsr' BEFORE importing it
# We find where basicsr is installed using pip show, avoiding the import crash
try:
    # Get the installation path of basicsr
    result = subprocess.run(['pip', 'show', 'basicsr'], capture_output=True, text=True)
    location_line = [line for line in result.stdout.split('\n') if line.startswith('Location:')][0]
    site_packages_path = location_line.split(': ')[1].strip()

    # Target the specific broken file
    target_file = os.path.join(site_packages_path, 'basicsr', 'data', 'degradations.py')

    if os.path.exists(target_file):
        print(f"Patching basicsr at: {target_file}")
        # Use sed to replace the broken import line
        !sed -i 's/from torchvision.transforms.functional_tensor import rgb_to_grayscale/from torchvision.transforms.functional import rgb_to_grayscale/' "{target_file}"
        print("Patch applied successfully.")
    else:
        print("Warning: Could not locate degradations.py to patch.")

except Exception as e:
    print(f"Patching failed: {e}")

# 4. Verify Import and GPU
try:
    import basicsr
    print("BasicSR imported successfully!")
except ImportError as e:
    print(f"BasicSR still failing: {e}")

assert torch.cuda.is_available(), "GPU not detected. Please change runtime to GPU"
print(f"Setup complete. Using GPU: {torch.cuda.get_device_name(0)}")

Setting up environment...
Cloning into 'Real-ESRGAN'...
remote: Enumerating objects: 759, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 759 (delta 106), reused 99 (delta 99), pack-reused 638 (from 1)
Receiving objects: 100% (759/759), 5.38 MiB | 12.72 MiB/s, done.
Resolving deltas: 100% (415/415), done.
/content/Real-ESRGAN/Real-ESRGAN/Real-ESRGAN/Real-ESRGAN
Installing dependencies...
Patching basicsr at: /usr/local/lib/python3.12/dist-packages/basicsr/data/degradations.py
Patch applied successfully.
BasicSR imported successfully!
Setup complete. Using GPU: Tesla T4



# 2. Mount drive (optional)

In [1]:
from google.colab import drive
mount_drive=False #@param{type:"boolean"}

if mount_drive:
  drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


# 3. Upscale video

- The upscaled video will be saved to `output_dir`
- If google drive is mounted, it will be also saved at `MyDrive/Upscaled Videos (REAL-ESRGAN)`


In [ ]:
video_path="/content/gdrive/MyDrive/001-initial-config-test-get-repos.mov" #@param{type:"string"}
output_dir="/content/" #@param{type:"string"}
resolution = "FHD (1920 x 1080)" # @param ["FHD (1920 x 1080)", "2k (2560 x 1440)", "4k (3840 x 2160)","2 x original", "3 x original", "4 x original"] {type:"string"}
model = "RealESRGAN_x4plus" #@param ["RealESRGAN_x4plus" , "RealESRGAN_x4plus_anime_6B", "realesr-animevideov3"]

assert os.path.exists(video_path), "Video file does not exist"

video_capture = cv2.VideoCapture(video_path)
video_width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
video_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

final_width = None
final_height = None
aspect_ratio = float(video_width/video_height)

# Get output resolutions
match resolution:
  case "FHD (1920 x 1080)":
    final_width=1920
    final_height=1080
  case "2k (2560 x 1440)":
    final_width=2560
    final_height=1440
  case "4k (3840 x 2160)":
    final_width=3840
    final_height=2160
  case "2 x original":
    final_width=2*video_width
    final_height=2*video_height
  case "3 x original":
    final_width=3*video_width
    final_height=3*video_height
  case "4 x original":
    final_width=4*video_width
    final_height=4*video_height

if aspect_ratio == 1.0 and "original" not in resolution:
  final_height = final_width

if aspect_ratio < 1.0 and "original" not in resolution:
  temp = final_width
  final_width = final_height
  final_height = temp

scale_factor = max(final_width/video_width, final_height/video_height)
isEven = int(video_width * scale_factor) % 2 == 0 and int(video_height * scale_factor) % 2 == 0

# scale_factor needs to be even
while isEven == False:
  scale_factor += 0.01
  isEven = int(video_width * scale_factor) % 2 == 0 and int(video_height * scale_factor) % 2 == 0

print(f"Upscaling from {video_width}x{video_height} to {final_width}x{final_height}, scale_factor={scale_factor}")

!python inference_realesrgan_video.py -n {model} -i '{video_path}' -o '{output_dir}' --outscale {scale_factor}

video_name_with_ext = os.path.basename(video_path)
video_name = video_name_with_ext.replace(".mp4","")
upscaled_video_path = f"{output_dir}{video_name}_out.mp4"
final_video_name = f"{video_name}_upscaled_{final_width}_{final_height}.mp4"
final_video_path = os.path.join(output_dir, final_video_name)

# crop to fit
if "original" not in resolution:
  print("Cropping to fit...")
  command = f"ffmpeg -loglevel error -hwaccel cuda -y -i '{upscaled_video_path}' -c:v h264_nvenc -filter:v  'crop={final_width}:{final_height}:(in_w-{final_width})/2:(in_h-{final_height})/2' -c:v libx264 -pix_fmt yuv420p '{final_video_path}'"
  subprocess.run(command,shell=True)
else:
  # final video path = upscaled video path
  command = f"cp '{upscaled_video_path}' '{final_video_path}'"
  subprocess.run(command,shell=True)

print(f"Upscaled video saved to: {final_video_path}")

# save to drive
if mount_drive:
  drive_folder = "MyDrive/Upscaled Videos (REAL-ESRGAN)"
  save_directory_drive = f"/content/gdrive/{drive_folder}"
  os.makedirs(save_directory_drive, exist_ok=True)

  command = f"cp '{final_video_path}' '{save_directory_drive}/{final_video_name}'"
  subprocess.run(command,shell=True)
  print(f"Saved to drive: /{drive_folder}/{final_video_name}" )

!rm "{upscaled_video_path}"

Upscaling from 992x466 to 1920x1080, scale_factor=2.327596566523605
Downloading: "https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth" to /content/Real-ESRGAN/Real-ESRGAN/Real-ESRGAN/Real-ESRGAN/weights/RealESRGAN_x4plus.pth

100% 63.9M/63.9M [00:00<00:00, 357MB/s]
inference:  38% 70/186 [03:12<05:43,  2.96s/frame]

# 4. Disconnect runtime

In [ ]:
from google.colab import runtime

disconnect_when_finish = False  #@param{type:"boolean"}

if disconnect_when_finish:
  runtime.unassign()